# **LARS 求解**

## 1. Functools

###  1). Cholesky分解

In [ ]:
mchol <- function(x){
    mn <- dim(x)
    m <- mn[1]
    n <- mn[2]
  
    if(m != n) return("Wrong dimensions of matrix!")
    if(sum(t(x) != x) > 0) return("Input matrix is not symmetrical!")

    L <- matrix(0, m, m) 
    for(i in 1:m){    # 以列为单位求解L
        L[i,i] <- sqrt(x[i,i])
        if(i < m){
            L[(i+1):m,i] <- x[(i+1):m,i]/L[i,i]
            # 更新矩阵，便于下一次同样方法计算
            TLV <- L[(i+1):m,i]
            TLM <- matrix(TLV, m-i, m-i)
            TLM <- sweep(TLM, 2, TLV, "*")
            x[(i+1):m,(i+1):m] <- x[(i+1):m,(i+1):m] - TLM
        }
    }
    return(L)  
}

### 2). 变量退出

In [ ]:
gives <- function(mx, lmx){
    mc <- mx[1]/lmx 
    ms <- mx[2]/lmx
    matrix(c(mc,ms,-ms,mc),ncol=2)
}

In [ ]:
mgives <- function(L,k){
    p <- dim(L)[1]
    if(k > p) return("Wrong input of k!")
    Lk <- L[-k,]
    mk <- k
    while(mk<p){
        mx <- Lk[mk,mk:(mk+1)]
        lmx <- sqrt(sum(mx*mx))
        Lk[mk,mk:(mk+1)] <- c(lmx,0)
        if(mk < p-1) Lk[(mk+1):(p-1), mk:(mk+1)] <- Lk[(mk+1):(p-1), mk:(mk+1)] %*% gives(mx, lmx)
        mk <- mk + 1
    }
    return(Lk[,-p])
}

### 2). 变量进入

In [ ]:
# 回代法
mforwardsolve=function(L,b){
    mn <- dim(L)
    m <- mn[1]
    n <- mn[2]
    
    if(m != n) return("Wrong dimensions of matrix L!")
    if(m != length(b)) return("Wrong dimensions of matrix L or vector b!")
    x=rep(0,m)
    for(i in 1:m){
        x[i]=b[i]/L[i,i]
        if(i<m) b[(i+1):m] = b[(i+1):m]-x[i]*L[(i+1):m,i]   
    }
    return(x)  
}
mbacksolve=function(L,b){
  mn=dim(L); m=mn[1]; n=mn[2]
  if(m!=n) return ("Wrong dimensions of matrix L!")
  if(m!=length(b)) return ("Wrong dimensions of matrix L or vector b!") #检查输入参数是否符合要求
  x=rep(0,m)
  for(i in m:1){
    x[i]=b[i]/L[i,i]
    if(i>1) b[(i-1):1]=b[(i-1):1]-x[i]*L[(i-1):1,i]   #更新右端项   
  }
  x  
}

In [ ]:
forupdate <- function(L, xxk, xkxk){
    lk <- forwardsolve(L, xxk)
    lkk <- sqrt(xkxk - sum(lk*lk))
    return(as.matrix(rbind(cbind(L, 0), c(lk, lkk))))
}

### 3). 范数计算

In [ ]:
# 三个范数计算
l2norm <- function(x){
  return(sqrt(sum(x*x)))
}
l1norm <- function(x){
  return(sum(abs(x)))
}
l0norm <- function(x){
  return( sum(abs(x)>1e-13))
}

## 2. 准备数据

In [ ]:
#读取数据
library(ElemStatLearn)
data(prostate)
data <- prostate[,-10]
head(data)
np <- dim(data)
n <- np[1]
p <- np[2]-1
xn <- names(data)[1:p]
x <- as.matrix(data[,1:p])
xm <- apply(x,2,mean)
y <- data[,p+1]
ym <- mean(y)
## 中心化数据的xtx,xty
XTX <- t(x)%*%x - n*outer(xm,xm) # 中心化后的xtx
XTY <-t(x)%*%y - n*xm*ym
YY <- sum(y*y)-n*ym*ym

## 3. 编写choleskey模式的lars求解lasso函数

In [ ]:
myLars<-function(xtx,xty){
# x2-范数
uu2 <- diag(XTX)  # 向量（xi-ximean）^2
uu<-sqrt(uu2)  # 求每个变量的2范数
uu1<-1/uu  # 2范数的倒数
## 准备待求变量
reA <- NULL
relamb <- NULL
reRSS <- NULL
reb <- NULL
########################
A <- rep(F,8)
reA <- rbind(reA, A)
# 找到第一个活跃变量
j <- which.max(abs(XTY)*uu1) # 只对Y进行中心化，不进行标准化
B <- j # 活跃变量集
A[j] <- !A[j] # 选入活跃集
# 记录第一个lambda
lamb <- uu1[j]*abs(XTY[j])
relamb <- c(relamb, lamb)
# 初始化b为0
b <- rep(0,8)
reb <- rbind(reb ,b)
# 初始化的残差
RSS <- YY
reRSS <- c(reRSS,RSS)
# 初始化下三角矩阵
L <- matrix(uu[j],1,1) # mchol(xtx)只进行了中心化，没进行标准化，因此是uu[j]

#########################################

############################
while(TRUE){
  CC <- uu1*(XTY - XTX %*% b)
  SCC = sign(CC)
  SCCA <- SCC[A]
  SCCB <- SCC[B]
  XTXA <- XTX[A,A,drop=F]
  td <- as.matrix(mforwardsolve(L, uu[B]*SCCB),ncol =1)
  d <- as.matrix(mbacksolve(t(L),td),ncol = 1)
  
  a = uu1[-B] * XTX[!A,B] %*% d
  
  
  if(sum(A)<=0 | sum(A)>8 ){
    print("Something is wrong!")
    break
  }else{
    gam=rep(0,8)
    w <- -b[B]/d 
    gam[B] = ifelse( w>0 & w<lamb, w, lamb)
    if(sum(A)<8) gam[!A] =ifelse(a*lamb<=CC[!A], (lamb-CC[!A])/(1-a),(lamb+CC[!A])/(1+a))
      
    j = which.min(gam)
    gammin = gam[j]
    RSS <- RSS - 2*gammin*sum(XTY[B]*d) + 2*gammin*sum(b[A]*SCCA)+
      gammin^2*sum(d*SCCB) #更新RSS
    b[B] =b[B] + gammin * d
    lamb = lamb - gammin
    reA = rbind(reA, A)
    
    reRSS <- c(reRSS, RSS)
    
    relamb = c(relamb, lamb)
    reb =rbind(reb ,b)
    if(lamb==0) break
    
    #update L
    if(A[j] == FALSE){
      xxk <- XTX[B,j]
      xkxk <- uu2[j]
      B <- c(B,j) # 向活跃集中添加j
      L <- forupdate(L,xxk,xkxk) 
    }else{
      jj <- which(B==j)
      L <- mgives(L,jj)
      B <- B[-jj] # 从活跃集中扣除j
    }
    A[j] = !A[j]
  }
}

Cp <- reRSS/RSS*(n-p) - n + 2*apply(reA,1,sum)
re <- list(b=reb,lambda=relamb)
return (re)
}

## 4. 交叉验证

### 1). 原理介绍

将原始数据均分成k组，每个子集中的数据分别做一次验证集，其余k-1折数据作为训练集训练模型。我们的想法是，为了得到最优的模型，我们需要将每一个$\lambda$下的模型用k次不同的训练集进行训练，并在此训练集对应的测试集中求出MSE，将K次的MSE进行平均作为这个$\lambda$模型的预测误差。由于$\lambda$取值范围为$[0,\lambda_0]$，需要对这段区间进行离散化求出各个离散的$\lambda$值，对这些值分别进行交叉验证，比较它们的MSE，选择预测误差最小的那个作为我们的最终选择。
 
 **注意点**
 - 由于本次数据采用中心化的X,Y进行计算，如果先固定$\lambda$再对每一次（共k次）的训练集进行不同的中心化计算，计算量较大，因此倒过来思考，先固定计算出每一次中心化后的训练集和测试集，在用不同的$\lambda$训练模型
 - 对测试集中的数据进行中心化的时候，用的均值是训练集的均值
 - 用**插补**的方法计算出$\beta$

对每一折中的训练集，给定$\lambda_j$时，用写好的lars函数可以求出针对此训练集得出的$\lambda$路径向量和$\beta$矩阵，寻找给定的$\lambda_j$对应$\lambda$的哪个区间，然后找对应的$\beta$矩阵区间，因为$\beta$是线性变化的，因此可以用插补法得到$\lambda=\lambda_j$时lars模型在此训练集上的$\hat{\beta}$.用这个$\hat{\beta}$对预测集计算$SSE$，在此i折中遍历$\lambda_j$得到SSE向量。再遍历折数$k$，得到$SSE$矩阵（$k$行），对列求均值得到每个$\lambda_j$在$k$折交叉验证中得到的$MSE$。

### 2). 代码

In [ ]:
lassolarsCV = function(x, y, k=10){
  #source("lar.source.R")
  mlars <- function(XTX, XTY, lam=NULL){           ####lars输入为中心化以后的xtx,xty,lam为选定的lambda
    p <- nrow(XTX)                                    #当lam不为空时，迭代到lambda=lam时停止迭代，并输出估计系数
    w <- sqrt(diag(XTX))                              #当lam为空时，迭代到0，输出list(relamb,reb)
    w1 <- 1/w
    relamb <- NULL
    reb <- NULL
    #reA<-NULL
    #reVA<-NULL
    A <- rep(F,p)
    VA <- NULL  # 因为L的顺序是按照活跃变量活跃的顺序排的，所以几乎所有相关变量都要用到VA
    j <- which.max(abs(XTY)/w)
    A[j] <- !A[j]
    VA <- c(VA,j)
    #reA<-rbind(reA,A)
    #reVA<-rbind(reVA,c(VA,rep(NA,(p-length(VA)))))
    L <- matrix(w[j], 1, 1)
    lamb <- abs(XTY[j])*w1[j]
    b <- rep(0, p)
    relamb <- c(relamb, lamb)
    reb <- rbind(reb ,b)
    
    
    while(TRUE){
      if(sum(A)<=0 | sum(A)>p ){
        print("Something is wrong!")
        break
      }
      CC <- w1*drop(XTY - XTX %*% b)
      SCC <- sign(CC)
      SCCA <- SCC[VA]
      td <- forwardsolve(L,w[VA]*SCCA)# 因为L的顺序是按照VA排的，所以必须要用VA而不能用A
      d <- backsolve(t(L),td)  # d也是按照VA的顺序来的
      a <- drop(XTX[!A,VA,drop=F] %*% d) * w1[!A]# 必须要用VA而不能用A
      gam <- rep(0,p)
      ww <- -b[VA]/d # 由于d用了VA的顺序，所以ww必须要用VA而不能用A
      gam[VA] <- ifelse( ww>0 & ww<lamb, ww, lamb)# 由于ww用了VA的顺序，所以gam必须要用VA而不能用A
      if(sum(A)<p) gam[!A] <- ifelse( a*lamb<=CC[!A], (lamb-CC[!A])/(1-a), (lamb+CC[!A])/(1+a) )
      
      j <- which.min(gam)
      gammin <- gam[j]
      if((!is.null(lam))&&lamb-gammin<=lam){
        b[VA] <- b[VA] + (lamb-lam)*d
        return(b)
      }      
      b[VA] <- b[VA] + gammin*d # 去掉VA结果和正确的不一致
      lamb <- lamb - gammin
      relamb <- c(relamb, lamb)
      reb <- rbind(reb, b)
      if(lamb==0) break
      # 变量的进入和退出都要用到VA
      jj <- which(as.vector(VA)==j)
      if(length(jj)==0){#如果jj为空，变量进入
        XTXAJ <- XTX[VA,j,drop=T] 
        XTXJJ <- XTX[j,j]
        L <- forupdate(L,XTXAJ,XTXJJ) # 按照活跃的顺序依次在下面增加一行
        VA <- c(VA,j)
        #reVA<-rbind(reVA,c(VA,rep(NA,(p-length(VA)))))
      } else {#如果变量退出
        L <- mgives(L,jj)
        VA <- VA[-jj]
        #reVA<-rbind(reVA,c(VA,rep(NA,(p-length(VA)))))
      }
      A[j] <- !A[j]  # 更新活跃集（实际上只是想知道不活跃集）
      #reA<-rbind(reA,A)
    }
    list(relamb=relamb, reb=reb)                          
  } 
  np <- dim(x); n <- np[1]; p <- np[2]
  xn <- names(x)
  x <- as.matrix(x, nrow=n, ncol=p)
  xm <- apply(x,2,mean)
  ym <- mean(y)
  xtx <- t(x)%*%x - n*outer(xm,xm)
  xty <- drop(t(x)%*%y) - n*xm*ym

  index <- sample(c(rep(1:k, n%/%k), sample(1:k, n%%k)), n)
  ###########
  #lassolarsCV的私有属性，功能丰富时有待继续增加
  cv.b <- NULL; cv.lam <- NULL; cv.mmse <- NULL
  ############
  list(
    cv.choosemodel = function(){
      #####交叉验证选lambda的场合,输出最优估计系数以及CV_mse
      lars.full <- mlars(xtx, xty)
      lam.full <- lars.full$relamb#用于网格搜索的lambda
      b.full <- lars.full$reb
      lam.len <- length(lam.full)
      mse.len <- rep(0, lam.len)#各组数据拟合结果的mse加权求和
      #lam_search_i <- lam_search
      cv.mse <- 0
      for(i in 1:k){#k为交叉验证折数
        ni <- sum(index==i)
        n.cv.i <- n-ni
        xi <- x[index==i,,drop=FALSE]
        xmi <- apply( xi , 2, mean)
        if(ni>=p){
          xtxi <- t(xi)%*%xi - ni*outer(xmi, xmi)
        }else{
          xci <-  sweep(xi,2,xmi,"-")
          xtxi <- t(xci)%*%xci
        }

        yi <- y[index==i]#第i组数据后的均值
        ymi <- mean(yi)
        xtyi <- t(xi)%*%yi - ni*xmi*ymi

        xm.cv.i <- (n*xm-ni*xmi)/n.cv.i
        ym.cv.i <- (n*ym-ni*ymi)/n.cv.i

        xtx.cv.i <- xtx - xtxi - ni*n/n.cv.i*outer(xmi-xm,xmi-xm)#利用离差阵分解可推导此公式
        xty.cv.i <- xty - xtyi - (ni*n/n.cv.i)*(ymi-ym)*(xmi-xm)
        #将去掉第i组数据的xtx,xty传入lars得到迭代轨迹数据list(relamb,reb)
        lars.cv.i <- mlars(xtx.cv.i, xty.cv.i)


        b.cv.i <-lars.cv.i$reb
        lam.cv.i <- lars.cv.i$relamb
        #b0.cv.i <- ym.cv.i - drop(b.cv.i%*%xm.cv.i)
        if( lam.cv.i[1] < lam.full[1]){
          lam.cv.i <- c(lam.full[1], lam.cv.i)
          b.cv.i <- rbind(0, b.cv.i)
        }

        lam.len.cv.i <- length(lam.cv.i)
        xout <- apply(outer(lam.cv.i, lam.full[-lam.len],">="), 2, sum)
        dd <- (lam.cv.i[xout]-lam.full[-lam.len])/(lam.cv.i[xout]-lam.cv.i[xout+1])
        bapp.cv.i <- rbind( (b.cv.i[xout+1,]-b.cv.i[xout,])*dd + b.cv.i[xout,],
                            b.cv.i[lam.len.cv.i,])
        b0app.cv.i <- ym.cv.i - drop(bapp.cv.i%*%xm.cv.i)
        #xi%*%t(bapp.cv.i)
        cv.mse <- cv.mse + apply((sweep(xi%*%t(bapp.cv.i), 2, b0app.cv.i, "+")-yi)^2, 2, sum)
      }
      #giveup <- length(lam_search)-length(sum_mse)
      cv.mmse <- cv.mse/n
      minj <- which.min(cv.mmse)
      cv.lam <- lam.full[minj]#网格搜索中CV_mse最小的lambda
      cv.b <- b.full[minj,]#将此lambda用于全部数据重新建模
      cv.b0 <- ym-drop(cv.b%*%xm)#计算beta0
      cv.b <<- c(cv.b0, cv.b)
      names(cv.b) <<- c('inter',xn)
      #if(plot_cv){#如果要求画图,改变lam,CV_mse属性
      cv.mmse <<- data.frame(lambda=lam.full, cv.mase=cv.mmse)
    },
    output = function(){
      list(cv.mmse=cv.mmse, cv.b=cv.b)
    }
  )
  #lassolarsCV的方法（可从外部调用）
}


test <- lassolarsCV(x,y,97)
test$cv.choosemodel()
test$output()

lm(y~as.matrix(x))#对比最小二乘(不中心化不标准化有截距项)，与选择的lambda为0时lassolarsCV的输出结果相同
library(lars)#对比cv.lars
a = cv.lars(as.matrix(x), y, type='lasso', mode='step', K=97)#横坐标应该是abs(beta)/max(abs(bata)),我猜的
a